<a href="https://colab.research.google.com/github/marcoAkerman/M3ProcessamentoImagem/blob/main/TrabalhoM3PI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/dataset_completo.zip" -d /content/

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: /content/dataset/695239.png  
  inflating: /content/dataset/695239.txt  
  inflating: /content/dataset/695327.png  
  inflating: /content/dataset/695327.txt  
  inflating: /content/dataset/695462.png  
  inflating: /content/dataset/695462.txt  
  inflating: /content/dataset/695463.png  
  inflating: /content/dataset/695463.txt  
  inflating: /content/dataset/695522.png  
  inflating: /content/dataset/695522.txt  
  inflating: /content/dataset/695568.png  
  inflating: /content/dataset/695568.txt  
  inflating: /content/dataset/695569.png  
  inflating: /content/dataset/695569.txt  
  inflating: /content/dataset/695579.png  
  inflating: /content/dataset/695579.txt  
  inflating: /content/dataset/695580.png  
  inflating: /content/dataset/695580.txt  
  inflating: /content/dataset/695581.png  
  inflating: /content/dataset/695581.txt  
  inflating: /content/dataset/695606.png  
  inflating: /content/dataset/695606.t

Limpar test, train e val

In [ ]:
import shutil
import os

OUT = "/content/dataset_split"

# Apagar a pasta inteira se existir
if os.path.exists(OUT):
    shutil.rmtree(OUT)

# Criar novamente vazia
os.makedirs(OUT)
print("dataset_split esvaziado com sucesso!")


dataset_split esvaziado com sucesso!


In [ ]:
import os
import shutil
import random

DATASET = "/content/dataset"
OUT = "/content/dataset_split"

FRACTION = 0.05   # 📌 Use x% do dataset (altere se quiser)

# Criar diretórios
for split in ["train", "val", "test"]:
    os.makedirs(f"{OUT}/{split}/images", exist_ok=True)
    os.makedirs(f"{OUT}/{split}/labels", exist_ok=True)

def get_class_sequence(txt_path):
    seq = []
    with open(txt_path, "r") as f:
        for line in f:
            seq.append(int(line.split()[0]))
    return tuple(seq)

placas = {}

# 1. Varrer labels com imagem PNG correspondente
for file in os.listdir(DATASET):
    if file.endswith(".txt"):
        prefix = file[:-4]
        img_path = os.path.join(DATASET, prefix + ".png")

        if not os.path.exists(img_path):
            continue  # ignora labels sem imagem

        seq = get_class_sequence(os.path.join(DATASET, file))
        placas.setdefault(seq, []).append(prefix)

# 2. Aplicar redução proportional
placas_keys = list(placas.keys())
random.shuffle(placas_keys)

k = int(len(placas_keys) * FRACTION)
placas_keys = placas_keys[:k]

# 3. Recalcular splits
n = len(placas_keys)
train_keys = placas_keys[:int(0.7*n)]
val_keys   = placas_keys[int(0.7*n):int(0.85*n)]
test_keys  = placas_keys[int(0.85*n):]

def copy_files(prefix, split):
    img = prefix + ".png"
    txt = prefix + ".txt"
    shutil.copy(os.path.join(DATASET, img), f"{OUT}/{split}/images/{img}")
    shutil.copy(os.path.join(DATASET, txt), f"{OUT}/{split}/labels/{txt}")

# 4. Copiar arquivos
for key in train_keys:
    for p in placas[key]:
        copy_files(p, "train")

for key in val_keys:
    for p in placas[key]:
        copy_files(p, "val")

for key in test_keys:
    for p in placas[key]:
        copy_files(p, "test")

print("✔️ Dataset reduzido e split concluído!")


✔️ Dataset reduzido e split concluído!


In [ ]:
import glob

print("Train images:", len(glob.glob("/content/dataset_split/train/images/*")))
print("Train labels:", len(glob.glob("/content/dataset_split/train/labels/*")))
print("Val images:", len(glob.glob("/content/dataset_split/val/images/*")))
print("Val labels:", len(glob.glob("/content/dataset_split/val/labels/*")))


Train images: 4327
Train labels: 4327
Val images: 178
Val labels: 178


In [ ]:
data_yaml = """
train: /content/dataset_split/train/images
val: /content/dataset_split/val/images

nc: 36
names: [ '0','1','2','3','4','5','6','7','8','9',
         'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z' ]
"""

with open("/content/data.yaml", "w") as f:
    f.write(data_yaml)

print("data.yaml criado!")



data.yaml criado!


Treinar o YOLO

In [ ]:
!pip install ultralytics


In [ ]:
from ultralytics import YOLO


In [ ]:
model = YOLO("yolo11n.pt")


In [ ]:
results = model.train(
    data="/content/data.yaml",
    epochs=30,
    imgsz=512,
    batch=32,
    device=0  # usa GPU do Colab
)


Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrai

In [ ]:
from ultralytics import YOLO

# Carrega o modelo treinado
model = YOLO("/content/runs/detect/train4/weights/best.pt")

# Avalia no dataset de validação
results = model.val(data="/content/data.yaml", verbose=True)



Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,589,172 parameters, 0 gradients, 6.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 808.9±341.9 MB/s, size: 20.9 KB)
val: Scanning /content/dataset_split/val/labels.cache... 178 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 178/178 274.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 4.7it/s 2.5s
                   all        178       1246      0.862      0.939      0.953      0.811
                     1        101        101      0.982          1       0.99      0.782
                     2         62         87      0.983          1      0.995      0.846
                     3         24         24      0.962          1      0.995      0.845
                     4         92         92      0.992          1      0.995      0.828
                     5         46         

In [ ]:
# mean_results() retorna precision, recall, mAP50, mAP50-95
precision, recall, map50, map5095 = results.mean_results()
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"IoU (mAP50): {map50:.4f}")
print(f"IoU (mAP50-95): {map5095:.4f}")
print(f"F1-Score: {f1_score:.4f}")


Precision: 0.8622
Recall: 0.9388
IoU (mAP50): 0.9533
IoU (mAP50-95): 0.8115
F1-Score: 0.8989
